# **OJKGPT**

This is the Jupyter Notebook for the development of the 1st version of OJKGPT for the showcase demo capabilites to wealth team program. <br>
In this development, we use several data sources for enriching the RAG system of the chatbot. 

In [85]:
import nest_asyncio
nest_asyncio.apply()

In [86]:
from dotenv import load_dotenv
load_dotenv()

True

In [87]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [88]:
from utils.models_definer import ModelName

# storing or not
OCR_TRESHOLD = 0.98
TOP_K = 3
NUM_BATCH_EVAL = 20
STORE = True
DELETE = True
EVAL = True
model_name = ModelName.AZURE_OPENAI
model_name_eval = ModelName.OPENAI

## **LLM Models Define**

In [89]:
from llama_index.core import Settings
from utils.models_definer import get_llm_and_embedding

# ollama/openai/azure_openai
llm, embedding_llm = get_llm_and_embedding(model_name=model_name)

llm_eval = get_llm_and_embedding(model_name=model_name_eval)[0]


Settings.llm = llm
Settings.embed_model = embedding_llm

## **Loading Documents**

### **Document Reader and Node Parser**

In [90]:
from utils.documents_reader import read_documents
from utils.node_parser import parse_nodes

path = './data'

if STORE:
    docs = read_documents(path=path, ocr_treshold=OCR_TRESHOLD)
    nodes = parse_nodes(documents=docs, llm=llm)

MuPDF error: format error: cmsOpenProfileFromMem failed

Read 9 documents


## **Indexing & Storing**

In [91]:
from utils.index_store import (store_vector_index, load_vector_index, store_summary_index, load_summary_index)

# store
if STORE:
    vector_index = store_vector_index(nodes=nodes,embed_model=embedding_llm, delete=DELETE)
    summary_index = store_summary_index(nodes=nodes,llm=llm, delete=DELETE,embed_model=embedding_llm)
# load
else:
    vector_index = load_vector_index()
    summary_index = load_summary_index()

Deleted 35 index from the collection.


Generating embeddings: 100%|██████████| 35/35 [00:04<00:00,  7.26it/s]


Storing the vector index completed.


Summarizing documents:   0%|          | 0/9 [00:00<?, ?it/s]

current doc id: 79a620f6-8d03-403b-b9ed-960d1cbfa489


...

current doc id: 9977e68e-5e58-4be6-af21-7243b761b9d4


Summarizing documents:  89%|████████▉ | 8/9 [00:26<00:03,  3.10s/it]

current doc id: 1a3e0ef6-3db2-4212-8561-dd18762eaa5e


Generating embeddings: 100%|██████████| 9/9 [00:01<00:00,  5.49it/s]

Storing the summary index completed.


## **Querying**

### **Retriever**

In [92]:
from llama_index.core.indices.document_summary import (
    DocumentSummaryIndexLLMRetriever
)
from llama_index.core.retrievers import RecursiveRetriever

vector_retriever = vector_index.as_retriever(similarity_top_k=TOP_K)

recursive_retriever = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": vector_retriever}
)

summary_retriever = DocumentSummaryIndexLLMRetriever(
    index=summary_index,
    choice_top_k=TOP_K,
    # embed_model=embedding_llm,
    llm=llm,
)

### **Query Engine**

In [93]:
from llama_index.postprocessor.colbert_rerank import ColbertRerank

colbert_reranker = ColbertRerank(
    top_n=5,
    model="colbert-ir/colbertv2.0",
    tokenizer="colbert-ir/colbertv2.0",
    keep_retrieval_score=True,
)

In [94]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core import get_response_synthesizer
import time

vector_query_engine = RetrieverQueryEngine.from_args(retriever=recursive_retriever,llm=llm, node_postprocessors=[colbert_reranker])

response_synthesizer = get_response_synthesizer(response_mode="tree_summarize")
list_query_engine = RetrieverQueryEngine.from_args(retriever=summary_retriever,llm=llm_eval, node_postprocessors=[colbert_reranker], )

time.sleep(2)

In [95]:
a = "Apa judul peraturan 7/33/PBI/2005?" # Pencabutan atas Peraturan Bank Indonesia Nomor 5/17/PBI/2003 tentang Persyaratan dan Tata Cara Pelaksanaan Jaminan Pemerintah terhadap Kewajiban Pembayaran Bank Perkreditan Rakyat
b = "Kapan surat edaran No. 15/26/DPbS mulai berlaku?" # 1 Agustus 2013.
c = "Siapa nama dan jabatannya yang menandatangani surat dengan nomor 1/SEOJK.04/2013?" # NURHAIDA, kepala eksekutif pengawas pasar modal
d = "Saya ingin menyelenggarakan kegiatan pasar modal berikan saya nomor surat yang membahas mengenai hal ini!" # Peraturan Pemerintah Nomor 12 Tahun 2004
e = "Berapa persen jaminan moneter pada tanggal 20 Agustus 1958?" # 7,3%
f = "Surat edaran nomor berapa yang mengatur bank umum syariah dan unit usaha syariah?" # 15/26/DPbS
g = "Apa kepanjangan dari PAPSI?" # Pedoman Akuntansi Perbankan Syariah Indonesia
h = "apa judul peraturan nomor 112/KMK.03/2001?" # Keputusan Menteri Keuangan tentang Pemotongan Pajak Penghasil Pasal 21 atas Penghasilan berupa Uang Pesangon, Uang Tebusan Pensiun, dan Tunjangan Hari Tua atau Jaminan Hari Tua
i = "Saya ingin membuat sistem informasi lembaga jasa keuangan, berikan nomor regulasi dari peraturan yang membahas tentang manejemen risiko nya!" # 4/POJK.05/2021
j = "Apa kepanjangan dari SWDKLLJ?" # Sumbangan Wajib Dana Kecelakaan Lalu Lintas Jalan
k = "Berapa nilai SWDKLLJ mobil sedan?" # Rp. 140.000
l = "Apa latar belakang dari peraturan NOMOR 4/POJK.05/2021?" # dalam bentuk list
m = "Apa itu LJKNB?" # Lembaga Jasa Keuangan Non Bank
n = "Apakah KMK Nomor 462/KMK.04/1998 masih berlaku" # tidak
o = "Apa itu Uang Pesangon?" # penghasilan yang dibayarkan oleh pemberi kerja kepada karyawan dengan nama dan dalam bentuk apapun sehubungan dengan berakhirnya masa kerja atau terjadi pemutusan  hubungan kerja, termasuk uang penghargaan masa kerja dan uang  ganti kerugian
p = "Apa itu CKPN?" # Cadangan Kerugian Penurunan Nilai.
q = "Kapan, dimana, dan oleh siapa surat nomor PER- 06/BL/2012 ditetapkan?" # Surat nomor PER-06/BL/2012 ditetapkan pada tanggal 22 November 2012 di Jakarta oleh Ketua Badan Pengawas Pasar Modal dan Lembaga Keuangan.
r = "Apa kepanjangan PSAK?" # Pernyataan Standar Akuntansi Keuangan
s = "Apa itu 'shahibul maal'?" # Pemilik dana pihak ketiga

query_str = b

In [96]:
from llama_index.core.response.notebook_utils import display_response

response_vector = vector_query_engine.query(query_str)
display_response(response_vector)

**`Final Response:`** Surat edaran No. 15/26/DPbS mulai berlaku pada tanggal 10 Juli 2013.

In [97]:
response_summary = list_query_engine.query(query_str)
display_response(response_summary)

**`Final Response:`** Surat edaran No. 15/26/DPbS mulai berlaku pada tanggal 1 Agustus 2013.

## **Evaluation**

In [98]:
import os

# check directory exist
if not os.path.exists("./json_data"):
    os.makedirs("./json_data")

In [99]:
from llama_index.core.evaluation import (
    generate_question_context_pairs,
    EmbeddingQAFinetuneDataset,
)
if EVAL:
    if STORE:
        qa_dataset = generate_question_context_pairs(
            nodes=nodes,
            llm=llm_eval,
            num_questions_per_chunk=2,
        )
        qa_dataset.save_json("./json_data/pg_eval_dataset.json")
    else:
        qa_dataset = EmbeddingQAFinetuneDataset.from_json("./json_data/pg_eval_dataset.json")

100%|██████████| 35/35 [00:47<00:00,  1.36s/it]


### **Retrieval Evaluation**

In [100]:
metrics = ["hit_rate", "mrr", "precision", "recall", "ap", "ndcg"]

In [102]:
from utils.evaluator import get_retrieval_eval_df

if EVAL:
    vector_retrieval_eval_results = await get_retrieval_eval_df(name=f"Top-{TOP_K} Eval", metrics=metrics, retriever=vector_retriever, qa_dataset=qa_dataset)
    display(vector_retrieval_eval_results)

,retrievers,hit_rate,mrr,precision,recall,ap,ndcg
0,Top-3 Eval,0.9,0.690476,0.3,0.9,0.690476,0.349402


### **Response Evaluation**

In [103]:
from llama_index.core.evaluation import (
    FaithfulnessEvaluator,
    RelevancyEvaluator,
)

if EVAL:
    # model for evaluation
    relevancy_evaluator = RelevancyEvaluator(llm=llm_eval)
    faithfullness_evaluator = FaithfulnessEvaluator(llm=llm_eval)

#### **Faithfulness Evaluator**

Evaluate Response

In [104]:
from utils.evaluator import get_response_eval_df

if EVAL:
    vector_faithfullness_eval_result = faithfullness_evaluator.evaluate_response(response=response_vector, query=query_str)
    summary_faithfullness_eval_result = faithfullness_evaluator.evaluate_response(response=response_summary, query=query_str)
    display(get_response_eval_df(query=query_str, response=response_vector, eval_result=vector_faithfullness_eval_result))
    display(get_response_eval_df(query=query_str, response=response_summary, eval_result=summary_faithfullness_eval_result))

,Query,Response,Source,Evaluation Result,Reasoning
0,Kapan surat edaran No. 15/26/DPbS mulai berlaku?,Surat edaran No. 15/26/DPbS mulai berlaku pada tanggal 10 Juli 2013.,"spesifik, Bank wajib menyampaikan rencana tindak (action plan) yang akan dilakukan. D. Ketentuan mengenai estimasi penurunan nilai pembiayaan secara kolektif sebagaimana dimaksud dalam huruf B, merupakan acuan bagi Bank Syariah dalam menyusun dan menyajikan laporan keuangan serta menjadi acuan bagi Akuntan Publik dalam melakukan pemeriksaan laporan keuangan Bank Syariah. E. Hal-hal yang harus dilakukan oleh Akuntan Publik dalam pemeriksaan atas estimasi penurunan nilai kolektif adalah sebagai berikut: 1. Dalam pelaksanaan audit, Akuntan Publik bertanggung jawab untuk: a. menilai kewajaran penilaian sendiri (self-assessment) yang dilakukan oleh manajemen Bank Syariah dalam menetapkan keberadaan kondisi keterbatasan pengalaman kerugian spesifik sebagaimana dimaksud dalam Lampiran Surat Edaran Bank Indonesia ini; dan b. menilai kewajaran estimasi oleh manajemen Bank Syariah dalam menentukan penurunan nilai pembiayaan secara kolektif. 2. Apabila dalam pelak...",Pass,YES


,Query,Response,Source,Evaluation Result,Reasoning
0,Kapan surat edaran No. 15/26/DPbS mulai berlaku?,Surat edaran No. 15/26/DPbS mulai berlaku pada tanggal 1 Agustus 2013.,"C. Dengan diterbitkannya antara lain PSAK khusus tentang transaksi syariah, PSAK No. 50 (Revisi 2010) tentang Instrumen Keuangan: Penyajian, PSAK No. 55 (Revisi 2011) tentang Instrumen Keuangan: Pengakuan dan Pengukuran, dan PSAK No. 60 tentang Instrumen Keuangan: Pengungkapan, serta PSAK No.48 (Revisi 2009) tentang Penurunan Nilai Aset maka perlu dilakukan penyesuaian atas PAPSI 2003 menjadi PAPSI 2013 sebagaimana dimaksud pada Lampiran yang merupakan bagian tidak terpisahkan dari Surat Edaran Bank Indonesia ini. D. PAPSI 2013 merupakan pedoman dalam penyusunan dan penyajian laporan keuangan Bank Syariah. Untuk hal-hal yang tidak diatur dalam PAPSI 2013 tetap berpedoman kepada PSAK yang berlaku beserta pedoman pelaksanaannya sepanjang tidak bertentangan dengan prinsip syariah. II. PENGAKUAN PENDAPATAN DALAM TRANSAKSI JUAL BELI A. Sesuai dengan Fatwa Dewan Syariah Nasional Nomor: 84/DSN- MUI/XII/2012 tanggal 21 Desember 2012 tentang Metode Pengakuan Pen...",Pass,YES


Evaluate Source Nodes

In [105]:
from utils.evaluator import get_response_eval_sources_df

if EVAL:
    relevancy_eval_sources = get_response_eval_sources_df(query=query_str, response=response_vector, evaluator=faithfullness_evaluator)
    display(relevancy_eval_sources)

,Source,Metadata,Eval Result
0,"spesifik, Bank wajib menyampaikan rencana tindak (action plan) yang akan dilakukan. D. Ketentuan mengenai estimasi penurunan nilai pembiayaan secara kolektif sebagaimana dimaksud dalam huruf B, merupakan acuan bagi Bank Syariah dalam menyusun dan menyajikan laporan keuangan serta menjadi acuan bagi Akuntan Publik dalam melakukan pemeriksaan laporan keuangan Bank Syariah. E. Hal-hal yang harus dilakukan oleh Akuntan Publik dalam pemeriksaan atas estimasi penurunan nilai kolektif adalah sebagai berikut: 1. Dalam pelaksanaan audit, Akuntan Publik bertanggung jawab untuk: a. menilai kewajaran penilaian sendiri (self-assessment) yang dilakukan oleh manajemen Bank Syariah dalam menetapkan keberadaan kondisi keterbatasan pengalaman kerugian spesifik sebagaimana dimaksud dalam Lampiran Surat Edaran Bank Indonesia ini; dan b. menilai kewajaran estimasi oleh manajemen Bank Syariah dalam menentukan penurunan nilai pembiayaan secara kolektif. 2. Apabila dalam pelaksanaan audit, Akuntan Publik menemukan bahwa Bank Syariah tidak berada dalam kondisi keterbatasan pengalaman kerugian spesifik namun menerapkan estimasi penurunan nilai pembiayaan secara kolektif maka Bank Syariah dinilai tidak menerapkan PSAK 55 beserta pedoman pelaksanaannya dan melanggar Surat Edaran Bank Indonesia ini. 3. Temuan Akuntan Publik sebagaimana dimaksud dalam angka 2 harus diungkapkan oleh Akuntan Publik dalam laporan hasil audit dan Surat Komentar (Management Letter) dan wajib disampaikan kepada Bank Indonesia sebagaimana diatur dalam ketentuan Bank Indonesia yang mengatur mengenai transparansi kondisi keuangan bank. F. Dalam … F. Dalam rangka memberikan informasi yang lebih transparan kepada masyarakat dan pengguna laporan keuangan Bank, Bank Syariah yang menerapkan estimasi penurunan nilai pembiayaan secara kolektif wajib mengungkapkan informasi tersebut dalam Catatan atas Laporan Keuangan dalam Laporan Tahunan sebagaimana diatur dalam Surat Edaran Bank Indonesia mengenai laporan tahunan bank umum. VI. KETENTUAN PENUTUP Dengan berlakunya Surat Edaran Bank Indonesia ini, Surat Edaran Bank Indonesia Nomor 5/26/BPS tanggal 27 Oktober 2003 perihal Pelaksanaan Pedoman Akuntansi Perbankan Syariah Indonesia dinyatakan tidak berlaku bagi Bank Umum Syariah dan Unit Usaha Syariah. Surat Edaran Bank Indonesia ini mulai berlaku pada tanggal 1 Agustus 2013. Agar setiap orang mengetahuinya, memerintahkan pengumuman Surat Edaran Bank Indonesia ini dengan penempatannya dalam Berita Negara Republik Indonesia. Demikian agar Saudara maklum. BANK INDONESIA, HALIM ALAMSYAH DEPUTI GUBERNUR DPbS","{'file_name': 'ojk-sebi-15_26_dpbs-10072013-sebi_perihal_pelaksanaan_pedoman_akuntansi_perbankan_syariah_indonesia_sebi_26_pdf.pdf', 'title': 'Surat Edaran Bank Indonesia perihal Pelaksanaan Pedoman Akuntansi Perbankan Syariah Indonesia', 'sector': 'Perbankan', 'subsector': 'Perbankan Syariah', 'regulation_type': 'SEBI', 'regulation_number': '15/26/DPbS', 'effective_date': '10 Juli 2013', 'retrieval_score': 0.921248197555542}",Fail
1,"juta rupiah) sampai dengan Rp 100.000.000,00 (seratus juta rupiah) sebesar 10% (sepuluh persen); d. penghasilan bruto di atas Rp 100.000.000,00 (seratus juta rupiah) sampai dengan Rp 200.000.000,00 (dua ratus juta rupiah) sebesar 15% (lima belas persen); e. penghasilan bruto di atas Rp 200.000.000,00 (dua ratus juta rupiah) sebesar 25% (dua puliuh lima belas persen); Pasal 3 Pada saat Keputusan Menteri Keuangan ini mulai berlaku, Keputusan Menteri Keuangan Nomor 462/KMK.04/1998 dinyatakan tidak berlaku. Pasal 4 Keputusan ini mulai berlaku tanggal 1 Januari 2001. Agar setiap orang mengetahuinya, memerintahkan pengumuman Keputusan Menteri Keuangan ini dengan penempatannya dalam Berita Negara Republik Indonesia. Ditetapkan di Jakarta pada tanggal 6 Maret 2001 MENTERI KEUANGAN REPUBLIK INDONESIA PRIJADI PRAPTOSUHARDJO","{'file_name': 'ojk-peraturan_keputusan_mentri-112_kmk_03_2001-06032001-kmk_nomor_112_kmk_03_2001_112_kmk_03_2001_p

#### **Relevancy Evaluator**

Evaluate Response

In [106]:
if EVAL:
    relevancy_eval_result = relevancy_evaluator.evaluate_response(
        query=query_str, response=response_vector
    )

    display(get_response_eval_df(query=query_str, response=response_vector, eval_result=relevancy_eval_result))

,Query,Response,Source,Evaluation Result,Reasoning
0,Kapan surat edaran No. 15/26/DPbS mulai berlaku?,Surat edaran No. 15/26/DPbS mulai berlaku pada tanggal 10 Juli 2013.,"spesifik, Bank wajib menyampaikan rencana tindak (action plan) yang akan dilakukan. D. Ketentuan mengenai estimasi penurunan nilai pembiayaan secara kolektif sebagaimana dimaksud dalam huruf B, merupakan acuan bagi Bank Syariah dalam menyusun dan menyajikan laporan keuangan serta menjadi acuan bagi Akuntan Publik dalam melakukan pemeriksaan laporan keuangan Bank Syariah. E. Hal-hal yang harus dilakukan oleh Akuntan Publik dalam pemeriksaan atas estimasi penurunan nilai kolektif adalah sebagai berikut: 1. Dalam pelaksanaan audit, Akuntan Publik bertanggung jawab untuk: a. menilai kewajaran penilaian sendiri (self-assessment) yang dilakukan oleh manajemen Bank Syariah dalam menetapkan keberadaan kondisi keterbatasan pengalaman kerugian spesifik sebagaimana dimaksud dalam Lampiran Surat Edaran Bank Indonesia ini; dan b. menilai kewajaran estimasi oleh manajemen Bank Syariah dalam menentukan penurunan nilai pembiayaan secara kolektif. 2. Apabila dalam pelak...",Pass,YES


Evaluate Source Nodes

In [107]:
from utils.evaluator import get_response_eval_sources_df

if EVAL:
    relevancy_eval_sources = get_response_eval_sources_df(query=query_str, response=response_vector, evaluator=relevancy_evaluator)
    display(relevancy_eval_sources)

,Source,Metadata,Eval Result
0,"spesifik, Bank wajib menyampaikan rencana tindak (action plan) yang akan dilakukan. D. Ketentuan mengenai estimasi penurunan nilai pembiayaan secara kolektif sebagaimana dimaksud dalam huruf B, merupakan acuan bagi Bank Syariah dalam menyusun dan menyajikan laporan keuangan serta menjadi acuan bagi Akuntan Publik dalam melakukan pemeriksaan laporan keuangan Bank Syariah. E. Hal-hal yang harus dilakukan oleh Akuntan Publik dalam pemeriksaan atas estimasi penurunan nilai kolektif adalah sebagai berikut: 1. Dalam pelaksanaan audit, Akuntan Publik bertanggung jawab untuk: a. menilai kewajaran penilaian sendiri (self-assessment) yang dilakukan oleh manajemen Bank Syariah dalam menetapkan keberadaan kondisi keterbatasan pengalaman kerugian spesifik sebagaimana dimaksud dalam Lampiran Surat Edaran Bank Indonesia ini; dan b. menilai kewajaran estimasi oleh manajemen Bank Syariah dalam menentukan penurunan nilai pembiayaan secara kolektif. 2. Apabila dalam pelaksanaan audit, Akuntan Publik menemukan bahwa Bank Syariah tidak berada dalam kondisi keterbatasan pengalaman kerugian spesifik namun menerapkan estimasi penurunan nilai pembiayaan secara kolektif maka Bank Syariah dinilai tidak menerapkan PSAK 55 beserta pedoman pelaksanaannya dan melanggar Surat Edaran Bank Indonesia ini. 3. Temuan Akuntan Publik sebagaimana dimaksud dalam angka 2 harus diungkapkan oleh Akuntan Publik dalam laporan hasil audit dan Surat Komentar (Management Letter) dan wajib disampaikan kepada Bank Indonesia sebagaimana diatur dalam ketentuan Bank Indonesia yang mengatur mengenai transparansi kondisi keuangan bank. F. Dalam … F. Dalam rangka memberikan informasi yang lebih transparan kepada masyarakat dan pengguna laporan keuangan Bank, Bank Syariah yang menerapkan estimasi penurunan nilai pembiayaan secara kolektif wajib mengungkapkan informasi tersebut dalam Catatan atas Laporan Keuangan dalam Laporan Tahunan sebagaimana diatur dalam Surat Edaran Bank Indonesia mengenai laporan tahunan bank umum. VI. KETENTUAN PENUTUP Dengan berlakunya Surat Edaran Bank Indonesia ini, Surat Edaran Bank Indonesia Nomor 5/26/BPS tanggal 27 Oktober 2003 perihal Pelaksanaan Pedoman Akuntansi Perbankan Syariah Indonesia dinyatakan tidak berlaku bagi Bank Umum Syariah dan Unit Usaha Syariah. Surat Edaran Bank Indonesia ini mulai berlaku pada tanggal 1 Agustus 2013. Agar setiap orang mengetahuinya, memerintahkan pengumuman Surat Edaran Bank Indonesia ini dengan penempatannya dalam Berita Negara Republik Indonesia. Demikian agar Saudara maklum. BANK INDONESIA, HALIM ALAMSYAH DEPUTI GUBERNUR DPbS","{'file_name': 'ojk-sebi-15_26_dpbs-10072013-sebi_perihal_pelaksanaan_pedoman_akuntansi_perbankan_syariah_indonesia_sebi_26_pdf.pdf', 'title': 'Surat Edaran Bank Indonesia perihal Pelaksanaan Pedoman Akuntansi Perbankan Syariah Indonesia', 'sector': 'Perbankan', 'subsector': 'Perbankan Syariah', 'regulation_type': 'SEBI', 'regulation_number': '15/26/DPbS', 'effective_date': '10 Juli 2013', 'retrieval_score': 0.921248197555542}",Fail
1,"juta rupiah) sampai dengan Rp 100.000.000,00 (seratus juta rupiah) sebesar 10% (sepuluh persen); d. penghasilan bruto di atas Rp 100.000.000,00 (seratus juta rupiah) sampai dengan Rp 200.000.000,00 (dua ratus juta rupiah) sebesar 15% (lima belas persen); e. penghasilan bruto di atas Rp 200.000.000,00 (dua ratus juta rupiah) sebesar 25% (dua puliuh lima belas persen); Pasal 3 Pada saat Keputusan Menteri Keuangan ini mulai berlaku, Keputusan Menteri Keuangan Nomor 462/KMK.04/1998 dinyatakan tidak berlaku. Pasal 4 Keputusan ini mulai berlaku tanggal 1 Januari 2001. Agar setiap orang mengetahuinya, memerintahkan pengumuman Keputusan Menteri Keuangan ini dengan penempatannya dalam Berita Negara Republik Indonesia. Ditetapkan di Jakarta pada tanggal 6 Maret 2001 MENTERI KEUANGAN REPUBLIK INDONESIA PRIJADI PRAPTOSUHARDJO","{'file_name': 'ojk-peraturan_keputusan_mentri-112_kmk_03_2001-06032001-kmk_nomor_112_kmk_03_2001_112_kmk_03_2001_p

#### **Batch Evaluator (Faithfulness, Relevancy, Correctness)**

In [108]:
from llama_index.core.evaluation import BatchEvalRunner
from utils.evaluator import get_batch_eval_results
from utils.evaluator import get_batch_eval_df

if EVAL:
    runner = BatchEvalRunner(
        {
            "faithfulness": faithfullness_evaluator, 
            "relevancy": relevancy_evaluator,
        },
        workers=8,
    )

    vector_eval_results = await get_batch_eval_results(runner=runner, qa_dataset=qa_dataset, query_engine=vector_query_engine, num_queries=NUM_BATCH_EVAL)
    summary_eval_results = await get_batch_eval_results(runner=runner, qa_dataset=qa_dataset, query_engine=list_query_engine, num_queries=NUM_BATCH_EVAL)
    display(get_batch_eval_df(vector_eval_results))
    display(get_batch_eval_df(summary_eval_results))

Retrying llama_index.llms.openai.base.OpenAI._achat in 0.14772099952478102 seconds as it raised RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-Lw60FXF38dnbt4jdHY1w8iTH on tokens per min (TPM): Limit 60000, Used 58710, Requested 1548. Please try again in 258ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}.
Retrying llama_index.llms.openai.base.OpenAI._achat in 0.14772099952478102 seconds as it raised RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-Lw60FXF38dnbt4jdHY1w8iTH on tokens per min (TPM): Limit 60000, Used 58710, Requested 1548. Please try again in 258ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}.
Retrying llama_index.llms.openai.base.OpenAI._achat in 0.1477209995247

IndexError: list index out of range

## **Build Agent**

In [ ]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

query_engine_tools = [
    QueryEngineTool(
        query_engine=vector_query_engine,
        metadata=ToolMetadata(
            name="vector_tool",
            description=(
                f"Useful for retrieving specific context from the documents."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=list_query_engine,
        metadata=ToolMetadata(
            name="summary_tool",
            description=(
                "Useful for summarization questions related to the documents."
            ),
        ),
    ),
]

In [ ]:
system_prompt = """Anda adalah chatbot yang dapat membantu menjawab pertanyaan tentang berbagai jenis regulasi di Indonesia.

Anda HARUS selalu menjawab dengan BAHASA QUERY.
Anda TIDAK DIBENARKAN berimajinasi.
Anda HANYA DIBENARKAN menjawab pertanyaan berdasarkan dokumen yang telah Anda pelajari.
JANGAN MENGGUNAKAN informasi dari luar dokumen yang telah Anda pelajari.
Anda memiliki akses ke query tools untuk membantu Anda menemukan informasi yang relevan di basis data regulasi.
Berdasarkan informasi konteks dan bukan pengetahuan sebelumnya, jawablah pertanyaan HARUS menggunakan query tools yang tersedia.
Gunakan riwayat percakapan sebelumnya atau konteks di atas untuk berinteraksi dan membantu pengguna.

Anda memiliki akses ke query tools berikut: 
- **vector_tool**: Untuk menemukan konteks spesifik dari dokumen
- **summary_tool**: Untuk menemukan ringkasan dokumen

**Penjelasan Metadata:**
Metadata dokumen mencakup informasi berikut:
- **file_name**: Nama file dokumen
- **title**: Judul dokumen
- **sector**: Sektor yang dicakup oleh regulasi
- **subsector**: Subsektor yang dicakup oleh regulasi
- **regulation_type**: Jenis regulasi (misalnya, Surat Edaran OJK, Peraturan OJK)
- **regulation_number**: Nomor regulasi
- **effective_date**: Tanggal berlakunya regulasi

----------------------------------------------
Query: {query_str}
Jawaban:
"""

In [ ]:
from llama_index.core.storage.chat_store import SimpleChatStore
from llama_index.core.memory import ChatMemoryBuffer

chat_store = SimpleChatStore()
memory = ChatMemoryBuffer.from_defaults(
    token_limit=10000,
    chat_store=chat_store,
    chat_store_key="user1",
)

In [ ]:
from llama_index.core.callbacks import CallbackManager, LlamaDebugHandler

llama_debug = LlamaDebugHandler(print_trace_on_end=True)
callback_manager = CallbackManager([llama_debug])

In [ ]:
from llama_index.agent.openai import OpenAIAgent
from llama_index.core.agent.legacy.react.base import ReActAgent


# agent = OpenAIAgent.from_tools(
#     tools=query_engine_tools,
#     llm=llm,
#     verbose=True,
#     memory=memory,
#     system_prompt=system_prompt,
#     callback_manager=callback_manager,
# )

agent = ReActAgent.from_tools(
    llm=llm,
    memory=memory,
    tools=query_engine_tools,
    verbose=True,
    system_prompt=system_prompt,
    callback_manager=callback_manager,
)

In [ ]:
response_agent = agent.chat(message=query_str)

Thought: The current language of the user is Indonesian. I need to use a tool to help me answer the question.
Action: vector_tool
Action Input: {'input': 'surat nomor PER- 06/BL/2012'}
Observation: The given context does not provide any information about the content of the letter with the number PER-06/BL/2012.
Thought: I cannot answer the question with the provided tools.
Answer: Maaf, saya tidak dapat menjawab pertanyaan tersebut dengan informasi yang diberikan.
**********
Trace: chat
    |_llm -> 1.487115 seconds
    |_function_call -> 8.420747 seconds
    |_llm -> 0.640543 seconds
**********


In [ ]:
from llama_index.core.response.notebook_utils import display_response

display_response(response=response_agent)

**`Final Response:`** Maaf, saya tidak dapat menjawab pertanyaan tersebut dengan informasi yang diberikan.

In [ ]:
from llama_index.core.response.notebook_utils import display_source_node

for node in response_agent.source_nodes:
    display_source_node(node, show_source_metadata=True)    

**Node ID:** c5003300-7080-4dac-a1f4-0f079095e45a<br>**Similarity:** 0.7496330142021179<br>**Text:** KEMENTERIAN KEUANGAN REPUBLIK INDONESIA 
BADAN PENGAWAS PASAR MODAL DAN LEMBAGA KEUANGAN 
 
 
SAL...<br>**Metadata:** {'file_name': 'ojk-peraturan_bapepam-per_06_bl_2012-22112012-perubahan_atas_peraturan_ketua_bapepam_dan_lembaga_keuangan_tentang_kegiatan_perusahaan_pembiayaan_berdasarkan_prinsip_488_pdf.pdf', 'title': 'Perubahan atas Peraturan Ketua Badan Pengawas Pasar Modal dan Lembaga Keuangan Nomor PER-03/BL/2007 tentang Kegiatan Perusahaan Pembiayaan Berdasarkan Prinsip Syariah', 'sector': 'IKNB', 'subsector': 'IKNB Syariah', 'regulation_type': 'Peraturan Bapepam', 'regulation_number': 'PER-06/BL/2012', 'effective_date': '22 November 2012', 'retrieval_score': 0.9217435717582703}<br>

**Node ID:** e4fce92c-be24-455f-a3d5-61f585be9ccf<br>**Similarity:** 0.7489040493965149<br>**Text:** KEMENTERIAN KEUANGAN REPUBLIK INDONESIA 
BADAN PENGAWAS PASAR MODAL DAN LEMBAGA KEUANGAN 
 
- 2 -...<br>**Metadata:** {'file_name': 'ojk-peraturan_bapepam-per_06_bl_2012-22112012-perubahan_atas_peraturan_ketua_bapepam_dan_lembaga_keuangan_tentang_kegiatan_perusahaan_pembiayaan_berdasarkan_prinsip_488_pdf.pdf', 'title': 'Perubahan atas Peraturan Ketua Badan Pengawas Pasar Modal dan Lembaga Keuangan Nomor PER-03/BL/2007 tentang Kegiatan Perusahaan Pembiayaan Berdasarkan Prinsip Syariah', 'sector': 'IKNB', 'subsector': 'IKNB Syariah', 'regulation_type': 'Peraturan Bapepam', 'regulation_number': 'PER-06/BL/2012', 'effective_date': '22 November 2012', 'retrieval_score': 0.9235822558403015}<br>

**Node ID:** a0bf7f5c-3519-4d65-a612-6764ba20c9ef<br>**Similarity:** 0.6710928082466125<br>**Text:** MENTERI KEUANGAN
REPUBLIK INDONESIA
SALINAN
PERATURAN MENTERI KEUANGAN
36 /PMK.010/2008
NOMOR
TEN...<br>**Metadata:** {'file_name': 'ojk-klasifikasi_bapepam-36_pmk_010_2008-26022008-pmk_nomor_36pmk_010_tahun_2008_tentang_besar_santunan_dan_sumbangan_wajib_dana_kecelakaan_lalu_lintas_jalan_menas13_1389258036_pdf.pdf', 'title': 'Peraturan Menteri Keuangan Nomor 36/PMK.010/2008 tentang Besar Santunan dan Sumbangan Wajib Dana Kecelakaan Lalu Lintas Jalan', 'sector': 'IKNB', 'subsector': 'Asuransi', 'regulation_type': 'Klasifikasi Bapepam', 'regulation_number': '36/PMK.010/2008', 'effective_date': '26 Februari 2008', 'retrieval_score': 0.9205957055091858}<br>

In [ ]:
chat_store_string = chat_store.json()
loaded_chat_store = SimpleChatStore.parse_raw(chat_store_string)